In [40]:
print("Assistant Function Calling")

Assistant Function Calling


In [41]:
from openai import OpenAI
import time
import json


In [42]:
client: OpenAI = OpenAI()

Step 0: Define functions

In [43]:
def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location
    
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [44]:
def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [45]:
from openai.types.beta.assistant import Assistant
from typing import List,Dict

tools :List[Dict] = [
    {
        "type":"function",
        "function":{
            "name": "getNickname",
            "description": "Get the nickname of a city",
            "parameters":{
                "type":"object",
                "properties":{
                    "location":{
                        "type":"string",
                        "description":"The city or state"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "getCurrentWeather",
            "description": "Get the weather in location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                    "unit": {"type": "string", "enum": ["c", "f"]}
                        },
                "required": ["location"]
            }
        }
    }
]
     
assistant: Assistant = client.beta.assistants.create(
    instructions="You are a weather bot and provide user with answers",
    model="gpt-3.5-turbo-1106",
    name="Weather bot",
    tools=tools
)

In [46]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

In [51]:
from openai.types.beta.threads.thread_message import ThreadMessage

message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is temprature in tokyo"
)

In [52]:
show_json("ThreadMessage",message)

'ThreadMessage'

{'id': 'msg_6VQJYCFWzib1I7NP1VjcTxq7',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'What is temprature in tokyo'},
   'type': 'text'}],
 'created_at': 1701990915,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo'}

In [53]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,  
)

show_json("Run",run)

'Run'

{'id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
 'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701990918,
 'expires_at': 1701991518,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot and provide user with answers',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo',
 'tools': [{'function': {'name': 'getNickname',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city or state'}},
     'required': ['location']},
    'description': 'Get the nickname of a city'},
   'type': 'function'},
  {'function': {'name': 'getCurrentWeather',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city and state e.g. San Francisco, CA'},
  

In [54]:
available_functions = {
    "getNickname": getNickname,
    "getCurrentWeather": getCurrentWeather
}

while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    runSteps = client.beta.threads.runs.steps.list(thread_id=thread.id,run_id=run.id)
    show_json("Run Status", runSteps)
    
    if run.status == "requires_action":
        print("requires_action")

        if run.required_action.submit_tool_outputs and run.required_action.submit_tool_outputs.tool_calls:
            tools_outputs = []
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            for toolCall in tool_calls:
                function_name = toolCall.function.name
                function_args = json.loads(toolCall.function.arguments)
                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    response = function_to_call(**function_args)
                    tools_outputs.append(
                        {
                            "tool_call_id":toolCall.id,
                            "output": response
                        }
                    )
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tools_outputs
            )
        
    elif run.status == "completed":
        print("completed")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in reversed(messages.data):
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break
    elif run.status == "failed":
        print("failed")
        break
    elif run.status in ["queued","in_progress"]:
        print(run.status)
        time.sleep(5)
    else:
        print(f"Unexpected status: {run.status}")
        break

'Run Status'

{'data': [{'id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': None,
   'created_at': 1701990919,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'in_progress',
   'step_details': {'message_creation': None,
    'type': 'tool_calls',
    'tool_calls': [{'id': 'call_tplmPgZe2Ilp5PkX0mWHwagv',
      'type': 'function',
      'function': {'name': 'getCurrentWeather',
       'arguments': '{"location":"Tokyo","unit":"c"}'}}]},
   'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo',
   'type': 'tool_calls',
   'expires_at': 1701991518}],
 'object': 'list',
 'first_id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
 'last_id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
 'has_more': False}

in_progress


'Run Status'

{'data': [{'id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': None,
   'created_at': 1701990919,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'in_progress',
   'step_details': {'message_creation': None,
    'type': 'tool_calls',
    'tool_calls': [{'id': 'call_tplmPgZe2Ilp5PkX0mWHwagv',
      'type': 'function',
      'function': {'name': 'getCurrentWeather',
       'arguments': '{"location":"Tokyo","unit":"c"}'}}]},
   'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo',
   'type': 'tool_calls',
   'expires_at': 1701991518}],
 'object': 'list',
 'first_id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
 'last_id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
 'has_more': False}

...

{'data': [{'id': 'step_wOCC3dDPurD0ETVroSe0vRVr',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': None,
   'created_at': 1701990927,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'in_progress',
   'step_details': {'message_creation': {'message_id': 'msg_VKeVx5NkyJpohZdynXT55XzQ'},
    'type': 'message_creation'},
   'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo',
   'type': 'message_creation',
   'expires_at': 1701991518},
  {'id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': 1701990926,
   'created_at': 1701990919,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'completed',
   'step_details': {'

in_progress


'Run Status'

{'data': [{'id': 'step_wOCC3dDPurD0ETVroSe0vRVr',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': 1701990927,
   'created_at': 1701990927,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'completed',
   'step_details': {'message_creation': {'message_id': 'msg_VKeVx5NkyJpohZdynXT55XzQ'},
    'type': 'message_creation'},
   'thread_id': 'thread_IeJuODd3KQBiAGnmQpjqCNgo',
   'type': 'message_creation',
   'expires_at': None},
  {'id': 'step_dgjsR7YYieBOMK4MooHk3fIa',
   'assistant_id': 'asst_MZe4YICwxFRmVpLeDQhHMI8Y',
   'cancelled_at': None,
   'completed_at': 1701990926,
   'created_at': 1701990919,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_5SPvKIuwzoN1GujIdmHJuDxk',
   'status': 'completed',
   'step_details': {'to

completed
User: What is the nickname of tokyo

Assistant: The nickname of Tokyo is "TK".

User: What is temprature in tokyo

Assistant: The current temperature in Tokyo is 10°C.

